In [1]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

from datetime import datetime
from zoneinfo import ZoneInfo 
import matplotlib.pyplot as plt
import pandas_market_calendars as mcal
import pandas as pd
import os

bucket = os.getenv("S3_BUCKET", 'swpp-12-bucket')
path1 = f"s3://{bucket}/price-financial-info/year=2025/month=9/market=kospi/2025-09-29.parquet"
path2 = f"s3://{bucket}/company-profile/year=2025/month=9/market=kospi/2025-09-29.parquet"

df1 = pd.read_parquet(path1, engine="fastparquet")
df2 = pd.read_parquet(path2, engine="fastparquet")

kospi_df = pd.concat([df1, df2], axis=1).sort_values('market_cap', ascending=False)

In [26]:
for i in set(kospi_df['industry']):
    print(i, '|', len(kospi_df[kospi_df['industry']==i]))

금속 | 65
음식료·담배 | 47
농업, 임업 및 어업 | 2
전기·전자 | 81
오락·문화 | 14
비금속 | 25
보험 | 14
전기·가스 | 10
운송·창고 | 27
운송장비·부품 | 65
섬유·의류 | 31
부동산 | 26
증권 | 29
기타금융 | 97
의료·정밀기기 | 8
IT 서비스 | 26
기계·장비 | 33
화학 | 123
통신 | 5
제약 | 54
유통 | 71
종이·목재 | 20
일반서비스 | 37
은행 | 3
건설 | 38
기타제조 | 8


In [38]:
def get_stock_json(market_df, industry, max_rows):
    if industry=='전체': pass
    else: market_df = market_df[market_df['industry']==industry]

    if len(market_df)<max_rows: pass
    else: market_df = market_df.iloc[:max_rows]

    print('rows:', len(market_df))
    return market_df, market_df.reset_index().rename(columns={'index': 'ticker'}).to_json(orient="records", force_ascii=False)
    
market_df, market_df_json = get_stock_json(kospi_df, '금속', 100)
market_df.head()

rows: 65


name market industry   close  change  change_rate      market_cap  \
005490  POSCO홀딩스  KOSPI       금속  278500    3500         1.27  22539827132000   
010130      고려아연  KOSPI       금속  917000    1000         0.11  18361341341000   
079550    LIG넥스원  KOSPI       금속  507000   16000         3.26  11154000000000   
004020      현대제철  KOSPI       금속   34850    -300        -0.85   4650585607250   
103140        풍산  KOSPI       금속  121600   -1600        -1.30   3407752204800   

             BPS    PER   PBR      EPS   DIV      DPS       ROE  \
005490  732527.0  19.27  0.38  14451.0  3.59  10000.0  0.019728   
010130  403756.0  96.65  2.27   9488.0  1.91  17500.0  0.023499   
079550   55550.0  49.84  9.13  10173.0  0.47   2400.0  0.183132   
004020  143815.0   0.00  0.24      0.0  2.15    750.0  0.000000   
103140   80518.0  14.07  1.51   8642.0  2.14   2600.0  0.107330   

                                              explanation  
005490  동사는 1968년 설립되어 2022년 지주회사로 전환하였음. 철강부문은 자동차, 조...  
010130  동사는 1974년 설립된 종합비철금속 제련회사로 아연, 연, 금, 은, 동 등을 제...  
079550  동사는 1976년 설립된 금성정밀공업을 모태로 1998년 설립된 대한민국 대표 종합...  
004020  동사는 1953년 설립 이후 강원산업과 한보철강 자산 인수, 현대하이스코 합병을 통...  
103140  동사는 1968년 설립된 풍산홀딩스로부터 2008년 인적분할하여 설립되고, 미국과 ...

In [39]:
news = pd.read_json('business_top50.json')
news.head()

title  \
0               국가자원 화재 배터리, 보증기한 1년 지난 제품   
1             “이런 기업에 돈쭐 내줘야”…‘착한 K뷰티’ 기업들   
2          편의점4사, ‘혼추족’ 노린 명절 도시락 및 간편식 대전   
3               정부 전산망 마비에 금융위도 '비상대응반' 가동   
4  우체국 금융·택배 ‘올스톱’…정부24까지 멈춰서며 주말 일상 곳곳 마비   

                                                 url source  \
0         https://www.mk.co.kr/news/society/11430705   매일경제   
1        https://www.mk.co.kr/news/business/11430702   매일경제   
2        https://www.mk.co.kr/news/business/11430697   매일경제   
3  https://www.asiae.co.kr/article/20250927174904...  아시아경제   
4         https://www.mk.co.kr/news/society/11430689   매일경제   

               published_at                       fetched_at  \
0 2025-09-27 18:19:44+09:00 2025-09-27 19:05:45.297777+09:00   
1 2025-09-27 18:13:47+09:00 2025-09-27 19:05:45.297777+09:00   
2 2025-09-27 18:07:08+09:00 2025-09-27 19:05:45.297777+09:00   
3 2025-09-27 17:49:04+09:00 2025-09-27 19:05:45.297777+09:00   
4 2025-09-27 17:36:02+09:00 2025-09-27 19:05:45.297777+09:00   

                                             content  content_length  
0  2014년 8월 설치된 제품작년 점검서 문제 없었고내년 배터리 교체 예정“보증기한·...            1107  
1  사진 확대 서울 용산구에 위치한 아모레퍼시픽 사옥 전경. <아모레퍼시픽> 추석을 앞...            1065  
2  증가하는 명절 도시락 수요 겨냥4사4색 먹거리 출시에 가격할인도 사진 확대 CU가 ...            1504  
3  입력2025.09.27 17:49 시계아이콘00분 36초 소요 국가정보자원관리원(국...            2287  
4  사진 확대 지난 26일 국가정보자원관리원에서 발생한 화재로 인터넷 우체국 등 우편 ...            2738

In [40]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field
from typing import List

class StockPick(BaseModel):
    ticker: str = Field(description="선정한 종목의 티커. 반드시 제공된 종목 리스트 중 하나여야 함.")
    name: str = Field(description="선정한 종목의 이름. 반드시 제공된 종목 리스트와 일치해야 함.")
    news: List[str] = Field(
        description="종목 선정에 활용한 뉴스의 title 리스트. 반드시 제공된 뉴스 정보 JSON의 title 중 일부여야 함. 해당 뉴스가 없으면 종목명이 아닌, 빈 리스트([])로 반환할 것.")
    reason: List[str] = Field(description="종목 선정 이유를 자세히 기술.")

class StockPicks(BaseModel):
    picks: List[StockPick] = Field(
        default_factory=list,
        min_items=0,
        max_items=5,
        description="추천 종목 리스트(0~5개). 조건에 맞는 종목이 없으면 빈 리스트([])로 반환."
    )

risk_json = "공격투자형"  # 안정형, 위험중립형, 공격투자형
stocks_json = market_df_json
news_json = news.to_json(orient="records", force_ascii=False)

template = """
당신은 투자 리서치 애널리스트입니다.
아래 입력에 기반하여 사용자의 투자성향에 맞춰 0~5개의 종목을 선정하고,
다음 스키마에 맞춰 JSON만 반환하세요.

스키마:
- {{ "picks": [ {{ "ticker": str, "name": str, "news": List[str], "reason": List[str]}}, ... ] }}

규칙:
- 반드시 제공된 종목 리스트(stocks_json) 안에서만 선택하세요. 외부 지식 사용 금지.
- 뉴스는 제공된 목록(news_json)만 활용하세요.
- 선정 이유를 여러가지 측면에서 자세히 기술하세요.
- 사용자 투자성향(risk_json)을 반영하세요. (예: 리스크 선호 낮으면 변동성 낮고 재무 건전성 높은 종목 선호)
- 중복 종목은 금지.
- 조건에 맞는 종목이 전혀 없으면 picks는 빈 리스트([])로 반환하세요.
- 출력은 오직 스키마에 맞는 JSON으로만 하세요. 여는/닫는 말 없이.

[사용자 투자성향]
{risk_json}

[종목 정보(JSON records)]
{stocks_json}

[뉴스 정보(JSON records)]
{news_json}
"""

prompt = PromptTemplate.from_template(template)

# 리스트 스키마로 구조화 출력
llm = ChatOpenAI(model_name="gpt-5-nano").with_structured_output(StockPicks)

chain = prompt | llm
result: StockPicks = chain.invoke({"risk_json": risk_json, "stocks_json": stocks_json, "news_json": news_json})

In [41]:
for i in result.picks:
    print('ticker:', i.ticker)
    print('name:', i.name)
    print('news:', i.news)
    print('reasons:', i.reason)
    print('============================================')

ticker: 112610
name: 씨에스윈드
news: ['미래에셋, 글로벌 ETF 순자산 250조 돌파', '포스코, 中 거점 팔고 인니로…동남아 철강시장 뚫는다', "관세 악재에 환율 급등…증시 랠리 '급제동' [HK영상]"]
reasons: ['글로벌 해상풍력 타워 공급 기업으로 성장 여력 큼', '유럽·미국·베트남 등 다국적 생산기지 확보로 공급망 리스크 분산', '장기계약 기반의 안정적 매출 창출 가능성 증가', '밸류에이션이 매력적(PER 저・PBR 적정수준)하고 성장 모멘텀이 여전함', '대형 글로벌 풍력 수요 증가에 따른 수혜 기대']
ticker: 058430
name: 포스코스틸리온
news: ['포스코, 中 거점 팔고 인니로…동남아 철강시장 뚫는다', "관세 악재에 환율 급등…증시 랠리 '급제동' [HK영상]", '미래에셋, 글로벌 ETF 순자산 250조 돌파']
reasons: ['저평가된 밸류에이션(PER 6.49, PBR 0.58)과 높은 배당성향(BIV 5.82)', '다양한 도금강판 제품군 및 자동차/가전 산업의 견조한 수요', '동남아 생산기지 확대로 글로벌 경쟁력 강화 가능', 'ESG 및 친환경 강재 전환 모멘텀에 따른 장기성장 기대', '배당수익 매력과 안정적 현금창출 구조']
ticker: 306200
name: 세아제강
news: ['포스코, 中 거점 팔고 인니로…동남아 철강시장 뚫는다', "관세 악재에 환율 급등…증시 랠리 '급제동' [HK영상]", '미래에셋, 글로벌 ETF 순자산 250조 돌파']
reasons: ['배관용·유정용 강관 등 고부가가치 제품에 강점', '64년 노하우와 글로벌 생산망으로 안정적 공급능력 확보', '저평가된 밸류주로 포트폴리오 다각화에 유리', 'R&D 및 설비투자를 통한 품질 및 생산성 경쟁력 상승', '건설·에너지 분야 수요 증가에 따른 성장성 확보']
ticker: 460860
name: 동국제강
news: ['포스코, 中 거점 팔고 인니로…동남아 철강시장 뚫는다', "관세 악재